# <span style="color:#BCE7FD">Python pour le Big Data </span>
Par Chloé Magnier et Axelle Lorin, IMAGE

## <span style="color:#BCE7FD">Introduction</span>

Lorsque l'on aborde la  question du bitcoin, beaucoup visualisent de suite des rateliers remplis de cartes graphiques moulinant à tout va - beaucoup de recherche a déjà été accordée à la question de "comment optimiser une ferme de bitcoins".

Mais les acteurs de la blockchains ne sont pas tous à la tête de véritables usines - miner est accessible à tous. Malheureusement, compléter un bloc afin d'être rémunéré s'avère souvent plus complexe.

Comment faire beaucoup avec peu? Existe-t-il une manière d'optimiser ses revenus en minant avec des ressources très limitées?

C'est à ces dernières questions que nous allons tenter de répondre ici, en analysant une à une puis combinées trois dimensions sur le marché : les zones chaudes, les zones rentables et les périodes désertiques.

Une <span style="color:#F2545B">**zone chaude**</span> représente un site ou des acteurs avec *beaucoup de transactions*. Beaucoup de transactions -> plus de chances de compléter un bloc.

Une <span style="color:#4E937A">**zone rentable**</span> est un site avec des *fee élevés*, ou particulèrement intéressants. Fee élevé -> complétion de bloc plus intéressante

Une <span style="color:#F2C57C">**période désertique**</span> est une période avec *peu de miners*. Peu de miners -> moins de compétition pour compléter les blocs.

Nous allons donc tenter ici de déterminer si des espaces de ce type peuvent être dégagés, et analyser au passage les données pouvant ressortir de ces recherches, afin de trouver des pistes de réponses à la question suivante : 

**Pour les miners disposants de peu de ressources, est-il possible d'optimiser son revenu en minant à des périodes stratégiques?**

## <span style="color:#BCE7FD">Extraction des données et librairies</span>

In [40]:
# Chargement des librairies
import pandas as pd
import numpy as np
import plotly.express as px
import glob, os

In [41]:
# Chargement des données
global_data = pd.read_csv("./DefiEGC2024/timeseries/global.csv", index_col=0)
blockchain_actor = pd.read_csv("./DefiEGC2024/timeseries/blockchain_by_actor.csv")
networks = pd.concat(map(pd.read_csv, glob.glob(os.path.join("./DefiEGC2024/networks/", "*.csv"))), ignore_index= True)

## <span style="color:#F2C57C"> Périodes désertiques </span>
### Existe-t-il des périodes (jour de la semaine, mois de l'année) où moins de personnes minent?

In [42]:
# Modification des données
global_data_rentable = global_data
global_data_rentable["rolling_mean_mining"] = global_data["nb_mining"].rolling(7, center=True).mean()
global_data_rentable["date"] = global_data_rentable.index.astype("datetime64[ns]")

In [43]:
# Nombres de mineurs par jour de 2015 à 2017
fig1 = px.line(global_data, x=global_data.index, y="rolling_mean_mining")
fig1.update_layout(
    title="Nombre de mineurs par jour",
)

Visualisons d'abord la quantité moyenne de mineurs par jour de 2015 à 2017 : on remarque que l'on peut passer du quitte au double, d'à peine 130 à plus de 170 personnes concurrentes. De grand pics se distinguent ici et là, et la forme globale de la courbe semble être vaguement sinusoïdale (pics en janvier 2016 et 2017, chutes en mai 2015, juillet 2016 et 2017). Ces motifs sont-ils récurrent au cours d'une année? D'un mois?

In [44]:
# Regroupement des données par mois
global_data_by_date = global_data.groupby([global_data["date"].dt.year, global_data["date"].dt.month]).mean()
global_data_by_date.drop(columns=["year", "month"], inplace=True)
global_data_by_date.rename_axis(["year", "month"], inplace=True)
global_data_by_date.reset_index(inplace=True)
global_data_by_date["date"] = (global_data_by_date["year"].astype(str) + "-" + global_data_by_date["month"].astype(str) + "-01").astype("datetime64[ns]")

In [45]:
#Graphe nombre moyen mineurs 2015-2017
fig2 = px.line(global_data_by_date, x="date", y="rolling_mean_mining")
fig2.update_layout(
    title = "Nombre de mineurs par mois de 2015 à 2017",
)

In [46]:
#Nombre moyen de mineurs par mois
global_data["date"] = global_data.index.astype("datetime64[ns]")
global_data_by_date = global_data.groupby([global_data["date"].dt.month]).mean()
global_data_by_date.rename_axis(["month"], inplace=True)

fig1 = px.line(global_data_by_date, y="rolling_mean_mining")
fig1.update_layout(
    title="Nombre moyen de mineurs par mois",
) 

Voici maintenant le nombre moyen de mineurs chaque mois de l'année - on remarque tout de suite une baisse assez prononcée autour du mois de juillet, avec une dizaine de mineurs de moins. Cette baisse pourrait être liée à plusieurs facteurs, comme la chaleur de l'été limitant le mining pour éviter la surchauffe, ou plus simplement des départs en vacance.

Il peut donc être plus intéressant de miner en juillet, ou bien en avril et octobre où d'autres creux se manifestent. Ne disposant malheureusement que de deux ans de données, attention, ces résultats peuvent être biaisés ou non représentatifs, surtout au vu de la relativement faible différence d'un mois à l'autre.


Afin de se faire une idée plus précise, il faudrait avoir accès à des données sur des périodes de temps plus longues, de 2017 à nos jours par exemple, afin de voir si les tendances remarquées ici sont toujours pertinentes de nos jours.

In [47]:
# Nombre moyen de mineurs par jour de la semaine
global_data["date"] = global_data.index.astype("datetime64[ns]")
global_data_by_date = global_data.groupby([global_data["date"].dt.weekday]).mean()
global_data_by_date.rename_axis(["weekday"], inplace=True)

fig1 = px.line(global_data_by_date, y="rolling_mean_mining")
fig1.update_layout(
    title="Nombre de mineur par jour de la semaine",
) 

Pour les jours de la semaine, malheureusement, rien de concluant ne peut être extrait - on pourrait croire à première vue le samedi plus intéressant pour miner, mais la variation du nombre de miners est en réalité minime, d'à peine 0,1 pour une valeur moyenne d'environ 150.

## <span style="color:#BCE7FD">Nombre de miners & satoshi</span>

Avant d'enchaîner sur le point suivant, les <span style="color:#4E937A">zones rentables</span> ou à fee élevés, nous avons voulu nous intéresser au croisement de ces catégories.

Il semblerait logique qu'il y ai plus de miners quand les sommes perçues pour la complétion d'un bloc sont plus élevées. Mais quelle est la relation entre ces deux variables? Est-il possible de trouver un point critique dans leur évolution, une fenêtre de valeur de fee à cibler, afin de maximiser les satoshi percues en minimisant le nombre de compétiteurs à la complétion?

In [48]:
# Comparaison nombre de mineurs/nombre de satoshi percu
fig2 = px.scatter(
    global_data, 
    x="nb_mining", 
    y="total_mining_satoshi",
    color="year",
)
fig2.update_layout(
    title="Nombre de mineurs en fonction des sommes perçues par les miners",
)

Il apparait que la relation entre les deux est tout à fait linéaire - il n'est donc à priori pas possible d'extraire un point critique comme discuté plus haut.

Cependant, la visualisation de ces données laisse apparaître une scission évidente au milieu de l'année 2016. A quoi cela peut donc être dû?

In [49]:
#Nombre de satoshi créées par jour de 2015 à 2017
global_data["rolling_paid_satoshi"] = global_data["total_mining_satoshi"].rolling(7, center=True).mean()
global_data["rolling_created_satoshi"] = global_data["newly_created_coins"].rolling(7, center=True).mean()
global_data["rolling_fee"] = (global_data["mean_fee_satoshi"]).rolling(7, center=True).mean()
fig3 = px.line(
    global_data, 
    y = ["rolling_paid_satoshi", "rolling_created_satoshi"]
)
fig3.update_layout(
    title="Nombre de satoshi crees par jour",
)


En cherchant un peu autour de la création de satoshi, nous pouvons extraire les données ci-dessus, qui viennent expliquer la tendance remarquée plus haut.

En effet, en juillet 2016, la quantité de satoshi créées a chuté drastiquement, impactant donc évidemment la quantité de satoshi perçues pour la complétion de blocs.

## <span style="color:#4E937A">Zones Rentables</span>
### Existe-t-il des acteurs ou périodes où les frais sont particulièrement intéressants?

In [50]:
#Mean fee for 100 par actor

montant_fee_actor = blockchain_actor.groupby("identity")["mean_fee_for100"].sum().sort_values(ascending=False)

px.bar(montant_fee_actor, title="Montant des fee par acteur")

Voici un histogramme représentant la moyenne des montants des transactions payés en frais par acteur - on remarque qu'une poignée se distingue, dont un particulièrement.

Il pourrait donc être intéressant de cibler en priorité les transactions effectués par ces acteurs.

Pour se faire une idée plus précise de quels acteurs sont particulièrement intéressants à cibler, il faudrait toutefois moduler ces chiffres par la quantité de transactions effectuées par ceux ci.

En effet, il est tout à fait possible que l'acteur 7227 n'ai effectué qu'une seule grosse transaction à un moment où les frais étaient plus élevés, et que le cibler ne s'avère au final pas particulièrement intéressant.

In [51]:
#Mean fee pour 100 transactions de 2015 à 2017
mean_fee = global_data["mean_fee_for100"]
px.line(mean_fee, title="Mean Fee for 100 Transactions by day")

Au niveau de la fréquence, il n'apparait pas de périodicité dans l'évolution des montants de transactions - on remarque deux pics bien marqués, mais rien de récurrent.

Il aurait pu être intéressant de regarder du côté des mois comme précédemment, afin de savoir si les transactions sont mieux payées en été ou au mois de septembre, mais visualiser les données comme nous venons de le faire indique que ce ne serait pas forcément pertinent : les résultats seraient bien trop biaisés par les valeurs des deux pics que nous voyons ici.

In [52]:
#Moyenne des mean fee pour 100 transactions par mois
global_data["date"] = global_data.index.astype("datetime64[ns]")
global_data_by_date = global_data.groupby([global_data["date"].dt.month]).mean()
global_data_by_date.rename_axis(["month"], inplace=True)

px.bar(global_data_by_date["mean_fee_for100"], title="Mean Fee for 100 Transactions by Month")

Si l'on y jette un regard quand même, on voit en effet distinctement l'impact des deux pics remarqués plus tôt.

Cependant, si l'on décide de faire abstraction de ces deux mois, il semble avoir un léger pic autour du mois de juin - avec des données s'étalant sur plus de mois, peut être remarquerions nous la même tendance, étalée aux mois de juillet et août. Peut être même pourrions nous la relier à la légère baisse en nombre de miners remarquée dans la section précédente?

## <span style="color:#F2545B">Zones Chaudes</span>
Existe-t-il des acteurs à l'origine de beaucoup de transactions, qu'il pourrait être intéressant de cibler? Et des périodes récurrentes avec beaucoup de transactions?

Pour les zones chaudes, nous allons étudier deux types d'acteurs : les sources et les targets.
Cette distinction est faite afin d'avoir l'oeil ouvert et de pouvoir dégager les différences entre les transactions reçues et celles envoyées.

In [53]:
# Nombre de transactions par acteur source
nb_transac_actor = blockchain_actor.groupby("identity")["nb_transactions"].sum().sort_values(ascending=False)

px.bar(nb_transac_actor, title="Nombre de transactions par acteur source")

In [54]:
# Nombre de transactions par acteur cible
nb_transac_target = blockchain_actor.groupby("identity")["nb_received"].sum().sort_values(ascending=False)

px.bar(nb_transac_target, title="Nombre de transactions par acteur cible")

On remarque de suite que, comme pour les fee par acteur, seule une poignée limitée d'acteurs se distingue, avec un nombre très élevé de transactions.
De plus, certains semblent envoyer bien plus qu'ils ne recoivent, ou vice-versa.


<span style="color:red">ATTENTION</span> : Il faut zoomer pour afficher les données sur le prochain graphe, il y a un impact très fort des acteurs qui ont un nombre de transactions élevé au début du graphe.

In [55]:
# Nombre de transactions par acteur source
nb_transac_actor_net = networks.groupby("Source")["nb_transactions"].sum().sort_values(ascending=False)

px.bar(nb_transac_actor_net, title="Nombre de transactions par acteur source")

In [56]:
# Nombre de transactions par acteur cible
nb_transac_target_net = networks.groupby("Target")["nb_transactions"].sum().sort_values(ascending=False)

px.bar(nb_transac_target_net, title="Nombre de transactions par acteur cible")

In [57]:
#nombre de transactions par jour de 2015 à 2017
blockcain_by_day = blockchain_actor.groupby("date")["nb_transactions"].sum()

px.line(blockcain_by_day, title="Nombre de transactions par jour")

On remarque ici deux énormes pics en 2015 dans le nombre de transactions - ceci viens expliquer les pics analogues remarqués dans les fees pour cette période.

En effet, nous n'avons pas remarqué de pic de la sorte dans le nombre de miners - le réseau s'est retrouvé du jour au lendemain à devoir gérer une quantité inouie de transactions, mais avec autant de miners et autant de ressources.

Pour tenir le coup, le montant des fee a donc été augmenté sur cette période, afin de signaler aux miners qu'il s'agissait d'une bonne période pour miner, et qu'il serait bon pour eux de débloquer plus de ressources de calcul pour le réseau.

Vérifions maintenant que ces deux variables sont correlées.

In [58]:
print("Corrélation mean_fee | nb_transaction sur l'intégralité des données :")
print(global_data[['mean_fee_for100', 'nb_transactions']].corr())
print("\n")
# corr is 0.16, not really meaningful - because day to day no impact, only correlated on big events creating peaks like above
# lets now check for correlation around the peaks, from July 2015 to October 2015
peaks = global_data[(global_data['year'] == 2015) & (global_data['month'] >= 7) & (global_data['month'] <= 10)]
print("Corrélation mean_fee | nb_transaction autour des pics isolés :")
print(peaks[['mean_fee_for100', 'nb_transactions']].corr())


Corrélation mean_fee | nb_transaction sur l'intégralité des données :
                 mean_fee_for100  nb_transactions
mean_fee_for100         1.000000         0.160474
nb_transactions         0.160474         1.000000


Corrélation mean_fee | nb_transaction autour des pics isolés :
                 mean_fee_for100  nb_transactions
mean_fee_for100         1.000000         0.633694
nb_transactions         0.633694         1.000000


Sur l'intégralité des données, il ne semble pas y avoir de corrélation entre le nombre de transactions et les frais. En effet, au jour le jour et globalement, les deux sont affectés par de nombreux facteurs différents, et pas forcément liées.

Cependant, en vérifiant autour des pics, on remarque que les deux variables évoluent bien de concert et sont corrélées - alors même qu'une bonne partie du tableau, correspondant à la période de retour au plat entre les pics, viens certainement atténuer le lien.

Cette corrélation et surtout son apparition seulement lors de moments critique nous permet de dégager des ouvertures intéressantes : pourrait-on prédire la hausse des frais de transactions et donc l'arrivée d'une période rentable en étudiant l'évolution du nombre de transactions?

In [59]:

px.line(peaks[['mean_fee_for100']], title="Pics isolés, mean_fee")

In [60]:

px.line(peaks[['nb_transactions']], title="Pics isolés, nb_transactions")

En affichant côte à côte les deux graphes, on remarque que le nombre de transactions semble en effet grimper quelques jours ou en même temps que les frais.

Avec plus de données sur des pics du genre ou bien à partir de fichiers de transactions contenant des informations de temps plus précises que ce à quoi nous avons accès (par exemple sur l'heure de la transaction), nous pourrions confirmer cette tendance, et peut être même mettre au point un algorithme capable de prédire de telles hausses.

## <span style="color:#BCE7FD">Conclusion</span>

Bien qu'il soit difficile de dégager de réelles périodes optimales de minage des analyses faites ici, nous en tirons tout de même une corrélation intéressante, entre la quantité de transactions et le montant des fees payés.

Même sans pouvoir créer d'algorithme permettant de détecter les pics rentables, une solution simple pour commencer à optimiser son minage avec peu de ressources serait donc de tracker la quantité de transactions effectuées en priorité, et la quantité de miners ensuite, afin d'être alerté à l'aube de pics comme ceux étudiés plus haut.